# IOOS models: how to find model data in the catalog

This is the third post on the series “IOOS Ocean Models IOOS.”

In [1]:
models = {
    "DOPPIO": {
        "RA": "MARACOOS",
        "url": "http://tds.marine.rutgers.edu/thredds/dodsC/roms/doppio/2017_da/avg/Averages_Best",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "NYHOPS": {
        "RA": "MARACOOS",
        "url": "http://colossus.dl.stevens-tech.edu:8080/thredds/dodsC/latest/Complete_gcmplt.nc",
        "var": {"standard_name": "sea_water_temperature"},
    },
    "NECOFS-GOM3": {
        "RA": "NERACOOS",
        "url": "http://www.smast.umassd.edu:8080/thredds/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_GOM3_FORECAST.nc",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "NECOFS-MASSBAY": {
        "RA": "NERACOOS",
        "url": "http://www.smast.umassd.edu:8080/thredds/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST.nc",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "CNAPS": {
        "RA": "SECOORA",
        "url": "http://thredds.secoora.org/thredds/dodsC/SECOORA_NCSU_CNAPS.nc",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "CMOP-SELFE": {
        "RA": "NANOOS",
        "url": "http://amb6400b.stccmop.org:8080/thredds/dodsC/model_data/forecast",
        "var": {"standard_name": "average_sea_water_temperature"},
    },
    "OSU-ROMS": {
        "RA": "NANOOS",
        "url": "http://ona.coas.oregonstate.edu:8080/thredds/dodsC/NANOOS/OCOS",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "Hawaii-ROMS": {
        "RA": "PacIOOS",
        "url": "http://oos.soest.hawaii.edu/thredds/dodsC/hioos/roms_forec/hiig/ROMS_Hawaii_Regional_Ocean_Model_best.ncd",
        "var": {"standard_name": "sea_water_potential_temperature"},
    },
    "WCOFS": {
        "url": "http://opendap.co-ops.nos.noaa.gov/thredds/dodsC/WCOFS/fmrc/Aggregated_7_day_WCOFS_Fields_Forecast_best.ncd",
        "var": {"standard_name": "sea_water_temperature"},
    },
    "WestCoastUCSC": {
        "url": "http://oceanmodeling.pmc.ucsc.edu:8080/thredds/dodsC/ccsra_2016a_phys_agg_zlevs/fmrc/CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd",
        "var": {"long_name": "potential temperature"},
    },
}

In [2]:
from owslib import fes


cf_names = set([list(model["var"].values())[0] for model in models.values()])

kw = dict(
    wildCard="*",
    escapeChar="\\",
    singleChar="?",
    propertyname="apiso:Subject",
)

or_filt = fes.Or(
    [
        fes.PropertyIsLike(literal=("*{val}*"), **kw)
        for val in cf_names
    ]
)

In [3]:
from datetime import datetime
from ioos_tools.ioos import fes_date_filter


start = datetime(2020, 1, 1)
stop = datetime(2020, 1, 8)

begin, end = fes_date_filter(start, stop)

In [4]:
bbox = [-145, 20, -45, 50]
crs = "urn:ogc:def:crs:OGC:1.3:CRS84"


bbox_crs = fes.BBox(bbox, crs=crs)

In [5]:
filter_list = [
    fes.And(
        [
            bbox_crs,
            begin,
            end,
            or_filt,
        ]
    )
]

In [6]:
from owslib.csw import CatalogueServiceWeb
from ioos_tools.ioos import get_csw_records


endpoint = "https://data.ioos.us/csw"
csw = CatalogueServiceWeb(endpoint, timeout=120)
csw = get_csw_records(csw, filter_list, esn="full")

In [7]:
print(csw.records)

OrderedDict()
